In [126]:
from openai import OpenAI
import numpy as np
import json
import faiss
import os
import re

In [108]:
# from langchain_openai import ChatOpenAI
# from dotenv import load_dotenv
# load_dotenv()

# # print("Key exists:", "OPENAI_API_KEY" in os.environ)  # Should be True
# # print("Key length:", len(os.getenv("OPENAI_API_KEY")))  # Should be ~51 chars

# llm = ChatOpenAI()
# llm.invoke("Hello, world!")

In [109]:
#load json files
def load_documents(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    return data

# Generate embeddings for each document
def get_embedding(text):
    response = client.embeddings.create(model = "text-embedding-ada-002", input = text)
    embedding_array = np.array(response.data[0].embedding)
    
    return embedding_array


In [110]:
with open("C:/Users/rshua/keys/secret_key.json", "r") as file:
    secrets = json.load(file)
    api_key = secrets["OPENAI_API_KEY"]

client = OpenAI(api_key = api_key)

In [111]:
# data = load_documents("C:/Users/rshua/fantasy-football-chatbot/nfl_season_stats_partial.json")
data = load_documents("nfl_season_stats_partial.json")

In [112]:
# Convert JSON to a list of text documents
documents = []

for week_data in data["Weeks"]:
    week = week_data["week"]
    for position, position_data in week_data["Positions"].items():
        for player_name, player_stats in position_data["Players"].items():
            text = f"Week: {week}, Position: {position}, Player: {player_name}, Team: {player_stats['team']}, "
            text += f"Dropbacks: {player_stats['dropbacks']}, Attempts: {player_stats['att']}, Completions: {player_stats['comp']}, "
            text += f"Yards: {player_stats['yds']}, TDs: {player_stats['tds']}, INTs: {player_stats['ints']}, "
            text += f"Sacks: {player_stats['sks']}, Rush Carries: {player_stats['rushCarries']}, Rush Yards: {player_stats['rushYds']}, "
            text += f"Rush TDs: {player_stats['rushTds']}, Fumbles: {player_stats['fumbles']}, Fantasy Points: {player_stats['fantasyPts']}"
            documents.append({"text": text})
            # print(week)

last_week = week
next_week = week + 1


In [113]:
# Create a list of text documents and their embeddings
# texts = [doc["text"] for doc in documents]
texts = []

for i in documents:
    text_content = i["text"]
    texts.append(text_content)

embeddings = np.array([get_embedding(text) for text in texts])

# Store embeddings in FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# dimension = embeddings.shape[1]
# index = faiss.IndexFlatL2(dimension)
# index.add(np.array([embeddings]))
# faiss.write_index(index, "vector_store.index")



In [114]:
print(texts)

['Week: 1, Position: Quarterback, Player: Josh Allen, Team: BUF, Dropbacks: 30, Attempts: 23, Completions: 18, Yards: 232, TDs: 2, INTs: 0, Sacks: 2, Rush Carries: 9, Rush Yards: 39, Rush TDs: 2, Fumbles: 1, Fantasy Points: 33.2', 'Week: 1, Position: Quarterback, Player: Baker Mayfield, Team: TB, Dropbacks: 34, Attempts: 30, Completions: 24, Yards: 289, TDs: 4, INTs: 0, Sacks: 1, Rush Carries: 3, Rush Yards: 21, Rush TDs: 0, Fumbles: 0, Fantasy Points: 29.7', 'Week: 1, Position: Quarterback, Player: Jayden Daniels, Team: WAS, Dropbacks: 34, Attempts: 24, Completions: 17, Yards: 184, TDs: 0, INTs: 0, Sacks: 2, Rush Carries: 16, Rush Yards: 88, Rush TDs: 2, Fumbles: 3, Fantasy Points: 28.2', 'Week: 1, Position: Quarterback, Player: Anthony Richardson, Team: IND, Dropbacks: 24, Attempts: 19, Completions: 9, Yards: 212, TDs: 2, INTs: 1, Sacks: 2, Rush Carries: 6, Rush Yards: 56, Rush TDs: 1, Fumbles: 0, Fantasy Points: 27.1', 'Week: 1, Position: Quarterback, Player: Lamar Jackson, Team: BL

In [115]:
# def retrieve_relevant_docs(query):
#     query_embedding = get_embedding(query)
#     query_embedding = query_embedding.reshape(1, -1)

#     distances, indices = index.search(query_embedding)

#     relevant_docs = []

#     for i in indices[0]:
#         relevant_docs.append(texts[i])
    
#     return relevant_docs

In [136]:
def extract_player_names(query):
	# This pattern looks for names in the format "First Last" or "Firstname Lastname"
	# It matches two or more consecutive words starting with capital letters
	name_pattern = r'\b([A-Z][a-z]+(?:\s[A-Z][a-z]+)+)\b'
	names = re.findall(name_pattern, query)
	return list(set(names))  # Remove duplicates


In [137]:
# Function to retrieve the most relevant documents
def retrieve_relevant_docs(query, top_k=None):
    if top_k is None:
        top_k = last_week  # Use the last_week variable as default top_k
    
    player_names = extract_player_names(query)
    
    if not player_names:
        # If no player names found, proceed with original behavior
        query_embedding = get_embedding(query).reshape(1, -1)
        distances, indices = index.search(query_embedding, top_k)
        return [texts[i] for i in indices[0]]
    
    relevant_docs = []
    
    for player_name in player_names:
        # Create a query focusing on the specific player
        player_query = f"Player: {player_name}"
        query_embedding = get_embedding(player_query).reshape(1, -1)
        
        distances, indices = index.search(query_embedding, top_k)
        
        # Get the top_k documents for this player
        player_docs = [texts[i] for i in indices[0]]
        relevant_docs.extend(player_docs)
    
    return relevant_docs







# def retrieve_relevant_docs(query, top_k = 6):
#     query_embedding = get_embedding(query)
#     query_embedding = query_embedding.reshape(1, -1)

#     distances, indices = index.search(query_embedding, top_k)

#     relevant_docs = []

#     for i in indices[0]:
#         relevant_docs.append(texts[i])

#     return relevant_docs





    # query_embedding = get_embedding(query)
    # distances, indices = index.search(query_embedding.reshape(1, -1), top_k)
    
    # return [texts[i] for i in indices[0]]



In [138]:
system_message = """
### Role & Purpose
You are an expert Fantasy Football AI assistant designed to provide data-driven insights, strategic advice, and statistical analysis for NFL player performance. Your responses must be:
- **Concise** but informative.
- **Factual** (grounded in retrieved data).
- **Actionable** (offer clear recommendations when asked).

---

### Core Responsibilities
1. **Statistical Summaries**
   - Always specify the week number when referencing data (e.g., "In Week 7, Player X had...").
   - If no data exists for a requested week/player, state: "No records found for [Player] in Week [X]."

2. **Lineup Advice (Start/Sit)**
   - Compare players using:
     - Recent performance (last 2-3 weeks)
     - Matchup difficulty (opposing defense strength)
     - Volume trends (targets/carries)
   - Example: "Start Player A over Player B due to consistent red-zone targets."

   - **Response Format:**
    [Player A] vs [Player B] (Week {X}):
    ▸ **Recent**: [A-PTS] vs [B-PTS] (Last 3 avg)
    ▸ **Matchup**: [A-DEF] (Rank) vs [B-DEF] (Rank)
    ▸ **Volume**: [A-TGT]/[A-CAR] vs [B-TGT]/[B-CAR]
    ---
    [Verdict]: "Start [Player] because [reason]."
    [Risk]: "[Potential downside]."

3. **Trade Recommendations**
   - Use terms like "Buy low" or "Sell high" based on value trends.
   - Highlight under/overperforming players with reasoning.

4. **Next-Week Projections (Week {next_week})**
   - Base predictions on:
     - Recent form (last 3 weeks)
     - Matchup context
     - Role changes (injuries, depth chart)
   - Include estimates for:
     - Passing: Yards, TDs, INTs
     - Rushing: Yards, TDs
     - Fantasy points (specify scoring format)

---

### Key Rules
- **Temporal Context:**
  - "Last week" = Week {last_week}, "Next week" = Week {next_week}.
  - Reject requests beyond Week {next_week}: "I can only project for Week {next_week}."

- **Data Integrity:**
  - Never guess stats. If unsure: "Insufficient data to answer."
  - For rookies/injured players: "Limited sample size; monitor practice reports."

- **Response Format:**
[Week X] [Player Name]:

Passing: [YDS]/[TDs]/[INTs]

Rushing: [YDS]/[TDs]

Fantasy: [PTS] (PPR/Standard)

[Insight]: "[Trend/Matchup Analysis]."
[Advice]: "[Start/Sit/Trade] because [reason]."
"""

# system_message = (
#     "You are a helpful Fantasy Football assistant specializing in providing fantasy football advice. You assist with:"
#     "Summarizing NFL player statistics based on retrieved information"
#     "Providing valuable line up suggestions like start/sit recommendations"
#     "Providing insightful trade advice"

#     "If asked about information from a specific week, only retrieve documents from that week and always mention the specific week number in your response.\n"

#     f"'Last week' refers to Week {last_week} of the season, and 'next week' refers to Week {next_week}.\n"

#     "If asked to predict performance for 'next week', base your prediction on the player's skill level, "
#     "recent performances, and overall season trends.\n"

#     "Your 'next week' predictions should include key metrics such as passing yards, passing touchdowns, rushing yards, "
#     "and total fantasy points. Ensure your estimates are reasonable based on past data.\n"
# )

In [139]:
user_query = "Should I start Bo Nix or Josh Allen next week?"

retrieved_docs = retrieve_relevant_docs(user_query)
context = "\n".join(retrieved_docs)


In [140]:
detected_names = extract_player_names(user_query)
print(detected_names)

['Bo Nix', 'Josh Allen']


In [141]:
print(retrieved_docs)

['Week: 5, Position: Quarterback, Player: Bo Nix, Team: DEN, Dropbacks: 32, Attempts: 27, Completions: 19, Yards: 206, TDs: 2, INTs: 0, Sacks: 3, Rush Carries: 8, Rush Yards: 9, Rush TDs: 1, Fumbles: 0, Fantasy Points: 23.1', 'Week: 6, Position: Quarterback, Player: Bo Nix, Team: DEN, Dropbacks: 41, Attempts: 33, Completions: 19, Yards: 216, TDs: 2, INTs: 1, Sacks: 2, Rush Carries: 6, Rush Yards: 61, Rush TDs: 0, Fumbles: 0, Fantasy Points: 21.7', 'Week: 8, Position: Quarterback, Player: Bo Nix, Team: DEN, Dropbacks: 40, Attempts: 37, Completions: 28, Yards: 284, TDs: 3, INTs: 0, Sacks: 2, Rush Carries: 5, Rush Yards: 4, Rush TDs: 1, Fumbles: 0, Fantasy Points: 29.8', 'Week: 4, Position: Quarterback, Player: Bo Nix, Team: DEN, Dropbacks: 26, Attempts: 25, Completions: 12, Yards: 60, TDs: 1, INTs: 0, Sacks: 0, Rush Carries: 5, Rush Yards: 3, Rush TDs: 0, Fumbles: 0, Fantasy Points: 6.7', 'Week: 7, Position: Quarterback, Player: Bo Nix, Team: DEN, Dropbacks: 33, Attempts: 26, Completions

In [142]:
# print(f"Retrieved Context:\n{context}")
# print("")
# print(retrieved_docs)

In [143]:
prompt = f"Use the following information to answer the query:\n\n{context}\n\nUser Query: {user_query}\n\nAnswer:"

completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
)

print(completion.choices[0].message.content)


Bo Nix vs Josh Allen (Week 9):
▸ **Recent**: Nix 21.5 (Last 3 avg) vs Allen 22.1 (Last 3 avg)
▸ **Matchup**: Nix's upcoming defense rank is not provided, but considering general performance trends vs Allen's upcoming defense rank is also not provided. Generally, consider the defensive prowess against QBs.
▸ **Volume**: Nix 39.3 dropbacks/game vs Allen 34 dropbacks/game in the last three games

---
[Verdict]: "Start Josh Allen over Bo Nix because of his consistently higher fantasy scoring potential and proven track record."
[Risk]: "Bo Nix has been improving and might outscore Allen if the latter faces a tough defense or if Nix's play continues to trend upward."


In [14]:
# completion = client.chat.completions.create(
#   model="gpt-4o-mini",
#   store=True,
#   messages=[
#     {"role": "user", "content": "How did Saquon Barkley do in the 2024 season?"}
#   ]
# )

# print(completion.choices[0].message);
